In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

Using TensorFlow backend.


## Call elmo from tensorflow hub

In [3]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

INFO:tensorflow:Using /var/folders/8w/6mb75l1j1j364ds6vmh57qth0000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/var/folders/8w/6mb75l1j1j364ds6vmh57qth0000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_0:0 from checkpoint b'/var/folders/8w/6mb75l1j1j364ds6vmh57qth0000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/b_cnn_0:0 from checkpoint b'/var/folders/8w/6mb75l1j1j364ds6vmh57qth0000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/CNN/b_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_1:0 fro

INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias:0 from checkpoint b'/var/folders/8w/6mb75l1j1j364ds6vmh57qth0000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/var/folders/8w/6mb75l1j1j364ds6vmh57qth0000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/projection/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/var/folders/8w/6mb75l1j1j364ds6vmh57qth0000gn/T/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Ce

In [4]:
data = pd.read_csv('data/spam.csv', encoding='latin-1')

In [5]:
y = list(data['v1'])
x = list(data['v2'])

In [6]:
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [7]:
def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

In [8]:
test = encode(le, ['ham', 'spam', 'ham', 'ham'])

In [9]:
untest = decode(le, test)

In [10]:
test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [11]:
untest

array(['ham', 'spam', 'ham', 'ham'], dtype='<U4')

In [12]:
x_enc = x
y_enc = encode(le, y)

In [13]:
x_train = np.asarray(x_enc[:5000])
y_train = np.asarray(y_enc[:5000])

In [14]:
x_test = np.asarray(x_enc[5000:])
y_test = np.asarray(y_enc[5000:])

In [15]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [16]:
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [17]:
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=32)
    model.save_weights('./elmo-model.h5')

Epoch 1/1
1696/5000 [=========>....................] - ETA: 25:49 - loss: 0.1451 - acc: 0.9528

KeyboardInterrupt: 

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./elmo-model.h5')  
    predicts = model.predict(x_test, batch_size=32)

In [ ]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [ ]:
from sklearn import metrics

In [ ]:
print(metrics.confusion_matrix(y_test, y_preds))

In [ ]:
print(metrics.classification_report(y_test, y_preds))